In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import re
# files from https://www.nasdaq.com/market-activity/stocks/screener
# files gotten on 2024-09-29

date = '2024-09-29'

In [ ]:
nasdaq = pd.read_csv('nasdaq_screener_nasdaq_'+date+'.csv')
nyse = pd.read_csv('nasdaq_screener_nyse_'+date+'.csv')
nyse['Exchange'] = 'NYSE'
nasdaq['Exchange'] = 'NASDAQ'
company_data = pd.concat([nyse,nasdaq],axis=0)
del(nyse,nasdaq)
company_data = company_data.rename(columns={'Symbol':'Ticker'})
# company_data.index=company_data['Symbol']
# nyse_and_nasdaq.drop(columns='Symbol',inplace=True)
# nyse_and_nasdaq.index.name='Ticker'
company_data

In [ ]:
odd = list({x for x in company_data['Ticker'] if (not (str(x).isalnum()))})
odd_with_arrow = list({x for x in company_data['Ticker'] if  (str(x).__contains__('^')) })
odd_no_arrow = list({x for x in company_data['Ticker'] if (not (str(x).isalnum()) and  not (str(x).__contains__('^')) )})
company_data

In [ ]:
# there seem to be a few types of odd tickers here, ones formatted like xxx^x and ones with xx/x in it and/or whitespace.
# it seems like all the rows with a ticker of the format "xxx^x" are something like prefered stock or some other nonsense
# the ones with / need to have the / swapped with - for the yfinance library to get their data
# and the whitespace needs to be removed. This does that
odd_cleaned = {x: re.split('[^a-zA-Z0-9]+', x)[0] for x in odd_with_arrow}
odd_cleaned.update({x: x.strip().replace('/', '-') for x in odd_no_arrow})
# and this updates the names in the dataframe
company_data['Ticker'] = company_data['Ticker'].replace(odd_cleaned)
company_data = company_data.reset_index()
company_data = company_data.drop(columns='index')
company_data

In [5]:
# 
company_data.index = company_data['Ticker']
company_data = company_data.drop(columns=['Ticker'])
# clean up invalid data
company_data = company_data[company_data.index.notna()]

In [ ]:
company_data

In [ ]:
# Rename columns to be more code friendly (underscores)
rename_dict = {
    'Name': 'Name',
    'Last Sale': 'Last_Sale',
    'Net Change': 'Net_Change',
    '% Change': 'Percent_Change',
    'Market Cap': 'Market_Cap',
    'Country': 'Country',
    'IPO Year': 'IPO_Year',
    'Volume': 'Volume',
    'Sector': 'Sector',
    'Industry': 'Industry',
    'Exchange': 'Exchange'
}
company_data = company_data.rename(columns=rename_dict)
company_data

In [8]:
company_data.to_csv('company_data_cleaned_'+date+'.csv')

In [ ]:
company_data.index

In [ ]:
raise SystemExit("Stop right there!")

### Only run the code below if it is CURRENTLY the last day you have stock price data for AFTER MARKET CLOSE

In [11]:
from IPython.display import clear_output
from tqdm import tqdm


In [ ]:
# getting company data from yfinance.
# this does three things: 
# -gives us updated market cap data
# -will throw errors (which are caught) if the company doesn't have data in yfinance
# tells us if its an 'equity' or something in the 'Quote_Type' columns

In [ ]:
company_data_yf = pd.DataFrame({
    'Market_Cap':[],
    'Quote_Type':[],
    'Shares':[],
    'Currency':[],
    'Exchange':[],
    'Time_Zone':[]
})
company_data_yf.index.name = 'Ticker'

display(company_data_yf)

# list of tickers from nasdaq download data
tickers = company_data.index

# progress bar/for loop
for ticker in tqdm(tickers, desc="Fetching company data"):
    print(f'Getting {ticker}')
    
    try:
        company_data_yf.loc[ticker, 'Market_Cap'] = yf.Ticker(ticker).fast_info['marketCap']
    except:
        pass
    try:
        company_data_yf.loc[ticker, 'Quote_Type'] = yf.Ticker(ticker).fast_info['quoteType']
    except:
        pass
    try:
        company_data_yf.loc[ticker, 'Shares'] = yf.Ticker(ticker).fast_info['shares']
    except:
        pass
    try:
        company_data_yf.loc[ticker, 'Currency'] = yf.Ticker(ticker).fast_info['currency']
    except:
        pass
    try:
        company_data_yf.loc[ticker, 'Exchange'] = yf.Ticker(ticker).fast_info['exchange']
    except:
        pass
    try:
        company_data_yf.loc[ticker, 'Time_Zone'] = yf.Ticker(ticker).fast_info['timezone']
    except:
        pass

    clear_output()

display(company_data_yf, clear=True)


In [14]:
company_data_yf.to_csv('company_data_yf_.csv')

### START HERE

In [2]:
company_data_yf = pd.read_csv('company_data_yf_'+date+'.csv',index_col=0)
company_data_yf

,Market_Cap,Quote_Type,Shares,Currency,Exchange,Time_Zone
Ticker,,,,,,
A,4.254178e+10,EQUITY,287328000.0,USD,NYQ,America/New_York
AA,1.014760e+10,EQUITY,258406000.0,USD,NYQ,America/New_York
AACT,6.768750e+08,EQUITY,62500000.0,USD,NYQ,America/New_York
AAM,4.401797e+08,EQUITY,43974000.0,USD,NYQ,America/New_York
AAN,3.170855e+08,EQUITY,31550800.0,USD,NYQ,America/New_York
...,...,...,...,...,...,...
ZURA,2.595610e+08,EQUITY,63774200.0,USD,NCM,America/New_York
ZVRA,3.641152e+08,EQUITY,52617800.0,USD,NMS,America/New_York
ZVSA,2.535112e+06,EQUITY,1074200.0,USD,NCM,America/New_York


In [3]:
print(company_data_yf['Quote_Type'].value_counts())
print()
print(company_data_yf['Currency'].value_counts())
print()
print(company_data_yf['Exchange'].value_counts())

Quote_Type
EQUITY        6317
ECNQUOTE         4
ETF              4
MUTUALFUND       3
Name: count, dtype: int64

Currency
USD    6317
Name: count, dtype: int64

Exchange
NYQ    2409
NCM    1592
NMS    1534
NGM     785
PNK       3
YHD       3
PCX       1
ASE       1
Name: count, dtype: int64


In [4]:
company_data_clean = pd.read_csv('company_data_cleaned_'+date+'.csv',index_col=0)
company_data_clean

,Name,Last_Sale,Net_Change,Percent_Change,Market_Cap,Country,IPO_Year,Volume,Sector,Industry,Exchange
Ticker,,,,,,,,,,,
A,Agilent Technologies Inc. Common Stock,$148.06,1.3200,0.90%,4.254173e+10,United States,1999.0,2212993,Industrials,Biotechnology: Laboratory Analytical Instruments,NYSE
AA,Alcoa Corporation Common Stock,$39.27,-0.0100,-0.025%,1.014502e+10,United States,2016.0,5629158,Industrials,Aluminum,NYSE
AACT,Ares Acquisition Corporation II Class A Ordina...,$10.8299,0.0199,0.184%,0.000000e+00,NaN,2023.0,2476,Finance,Blank Checks,NYSE
AAM,AA Mission Acquisition Corp. Class A Ordinary ...,$10.01,0.0000,0.00%,0.000000e+00,NaN,2024.0,537105,NaN,NaN,NYSE
AAN,Aarons Holdings Company Inc. Common Stock,$10.05,0.0200,0.199%,3.086634e+08,United States,2020.0,1425538,Consumer Discretionary,Diversified Commercial Services,NYSE
...,...,...,...,...,...,...,...,...,...,...,...
ZURA,Zura Bio Limited Class A Ordinary Shares,$4.07,0.1000,2.519%,2.735328e+08,Cayman Islands,NaN,393346,Health Care,Biotechnology: Biological Products (No Diagnos...,NASDAQ
ZVRA,Zevra Therapeutics Inc. Common Stock,$6.92,-0.3900,-5.335%,3.641151e+08,United States,NaN,1660748,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ
ZVSA,ZyVersa Therapeutics Inc. Common Stock,$2.36,0.0600,2.609%,2.428903e+06,United States,2022.0,53919,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ


In [5]:
# filters out companies that yahoo finance doesn't have historical data for
company_data_clean_filtered = company_data_clean.loc[company_data_yf.index,:]
company_data_clean_filtered.shape

(6666, 11)

In [6]:
unique_ticker_list = list(set(company_data_clean_filtered.index))
print(unique_ticker_list)
print(len(unique_ticker_list))

['VRNS', 'PMTS', 'JFBRW', 'ROG', 'STKH', 'SRZNW', 'BNTX', 'OCS', 'BDC', 'JVA', 'SHMDW', 'EXTO', 'CHRS', 'CRL', 'IBOC', 'PSTV', 'KPTI', 'VMEO', 'INNV', 'RMT', 'AS', 'LGVN', 'MFI', 'UTMD', 'SIGI', 'NVEC', 'VOR', 'TCBI', 'BFZ', 'NMRK', 'CNTB', 'TT', 'MITAW', 'SBSW', 'KDP', 'PIII', 'COOT', 'NXL', 'EL', 'HBANM', 'COCO', 'XWEL', 'OCX', 'SGA', 'MNDR', 'LFT', 'GBIO', 'GOGL', 'VSH', 'API', 'FMX', 'BCSA', 'KTOS', 'GECCI', 'CHRD', 'ENG', 'NMR', 'RNR', 'CABO', 'ANTX', 'QURE', 'HOTH', 'GNFT', 'BBUC', 'TGT', 'KNSL', 'ASRT', 'TRIB', 'ARM', 'WFG', 'CODI', 'QIPT', 'LECO', 'GYRO', 'FRST', 'ZJYL', 'TSLA', 'DIOD', 'AMRC', 'SHCO', 'BFRI', 'GDEV', 'CELZ', 'DATS', 'VIRT', 'EME', 'STCN', 'FEMY', 'BDSX', 'CENT', 'AZEK', 'MBINM', 'ALGT', 'ROAD', 'OGE', 'FSS', 'UPST', 'LEGN', 'CCNE', 'MHLD', 'IBTX', 'CKPT', 'SXTC', 'CARM', 'SMXT', 'KZR', 'DSGN', 'HFFG', 'HLF', 'KBH', 'AVXL', 'OABIW', 'URI', 'HLLY', 'LIXT', 'PTIXW', 'ARES', 'UFCS', 'CMBM', 'SEDA', 'DRTS', 'SMBC', 'RCS', 'ASGI', 'SFBC', 'AGNCM', 'USFD', 'RENE', 'C

In [7]:
# Function to keep rows with the most non-NaN values
def keep_most_non_na(group):
    # Count non-NaN values in each row
    non_na_count = group.notna().sum(axis=1)
    # Filter rows with the maximum count of non-NaN values
    return group[non_na_count == non_na_count.max()]

In [8]:
company_data_clean_filtered_unique = pd.DataFrame()
for ticker in unique_ticker_list:
    company_data_clean_filtered_unique = pd.concat([company_data_clean_filtered_unique,
                                                   keep_most_non_na(company_data_clean_filtered.loc[[ticker],:])],
                                                   axis=0)
company_data_clean_filtered_unique = company_data_clean_filtered_unique.sort_index()

In [9]:
# tickers that appear more than once
repeated_index = company_data_clean_filtered_unique.index[company_data_clean_filtered_unique.index.duplicated(keep=False)]

# subset of the DataFrame with only the repeated index values
subset_df = company_data_clean_filtered_unique.loc[repeated_index].drop_duplicates()

# gets list of duplicate tickers
duplicate_tickers = list(set(subset_df.index))

# removes them
company_data_clean_filtered_unique = company_data_clean_filtered_unique.drop(index=duplicate_tickers,errors='ignore')
display(company_data_clean_filtered_unique)


,Name,Last_Sale,Net_Change,Percent_Change,Market_Cap,Country,IPO_Year,Volume,Sector,Industry,Exchange
Ticker,,,,,,,,,,,
A,Agilent Technologies Inc. Common Stock,$148.06,1.3200,0.90%,4.254173e+10,United States,1999.0,2212993,Industrials,Biotechnology: Laboratory Analytical Instruments,NYSE
AA,Alcoa Corporation Common Stock,$39.27,-0.0100,-0.025%,1.014502e+10,United States,2016.0,5629158,Industrials,Aluminum,NYSE
AACG,ATA Creativity Global American Depositary Shares,$0.7302,0.0078,1.08%,2.336650e+07,China,2008.0,26100,Real Estate,Other Consumer Services,NASDAQ
AACT,Ares Acquisition Corporation II Class A Ordina...,$10.8299,0.0199,0.184%,0.000000e+00,NaN,2023.0,2476,Finance,Blank Checks,NYSE
AADI,Aadi Bioscience Inc. Common Stock,$1.93,0.0200,1.047%,4.750663e+07,United States,NaN,81024,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ
...,...,...,...,...,...,...,...,...,...,...,...
ZVRA,Zevra Therapeutics Inc. Common Stock,$6.92,-0.3900,-5.335%,3.641151e+08,United States,NaN,1660748,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ
ZVSA,ZyVersa Therapeutics Inc. Common Stock,$2.36,0.0600,2.609%,2.428903e+06,United States,2022.0,53919,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ
ZWS,Zurn Elkay Water Solutions Corporation Common ...,$35.66,0.5700,1.624%,6.095310e+09,United States,2012.0,1408735,Industrials,Industrial Machinery/Components,NYSE


In [10]:
company_data_clean_filtered_unique.to_csv('company_data_cleaned_2024-09-29.csv')